# Tarea 5

__Alumno:__ Eduardo Carrasco

__Curso:__ Aprendizaje Automático 2022 

__Programa:__ DISC

__Profesor:__ Gonzalo A. Ruz

__Ayudante:__ Carlos Sepúlveda

**Tema**: Keras (Convolutional Neural Networks CNN) - aplicado a Flowers Recognition

## Instrucciones
* La actividad es por grupo de capstone project
* Por favor responder en este mismo notebook
* Renombrar el archivo agregando su nombre, por ejemplo tarea5_Lepe_Contreras.ipynb
* Subir el archivo al link de entrega Trabajo en webcursos que será habilitado
* __Fecha de entrega:__ viernes 04 de noviembre, 23:55 hrs.

In [1]:
#pip install tensorflow

In [ ]:
## Librerias generales
import os
import matplotlib.pyplot as plt
%matplotlib inline

from time import time
from glob import glob
from numpy import zeros, array, argmax, vstack
from sklearn.metrics import confusion_matrix, classification_report

## Keras tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.optimizers import Adam

### Customized function

In [ ]:
def plot_history(history, width=12, height=6):
  """
  DESCRIPTION:
    History performance of the keras model
  
  INPUT:
    @param history: history of performance of fitted model
    @type history: tensorflow.python.keras.callbacks.History

  OUTPUT:
    A graphic
  """

  ## Metrics keys stored in tensorflow object
  keys = list(history.history.keys())

  ## Number of epoch used for fit the model
  epoch = range(1, len(history.epoch) +1)

  ## Check if validation set was used.
  withValidation = False
  for key in keys:
    if 'val' in key:
      withValidation = True

  ## Number of metrics 
  nMetrics = len(keys)
  if withValidation:
    nMetrics = nMetrics//2

  ## Plot-space instance
  plt.figure(figsize=(width, height))

  for i in range(nMetrics):
    plt.subplot(nMetrics, 1, i+1)

    ## Plot (train) metric value
    labelMetric = keys[i]
    metric = history.history[keys[i]]
    plt.plot(epoch, metric, 'o-', label=labelMetric)

    if withValidation:
      ## Plot (validation) metric value
      labelMetricVal = keys[i+nMetrics]
      metricVal = history.history[keys[i+nMetrics]]
      plt.plot(epoch, metricVal, 'o-', label=labelMetricVal)

    plt.xlim(epoch[0], epoch[-1])
    plt.legend()
    plt.grid()

  plt.xlabel('Epoch')
  plt.show()

**Data source:**

[Flowers classification](https://www.kaggle.com/alxmamaev/flowers-recognition) (Classification)

In [ ]:
if not os.path.exists("dataFlowers.zip"):
    ## Download files

    !wget https://github.com/adoc-box/Datasets/raw/main/flowers.zip
    !wget https://github.com/adoc-box/Datasets/raw/main/flowers.z01

    ## join files 
    !zip -F flowers.zip --out dataFlowers.zip

    ## remove some files
    !rm flowers.z01 flowers.zip

    ## un-compress joined zip file
    !unzip dataFlowers.zip
    
else:
    if os.path.exists('data'):
        !rm -r data
        !unzip dataFlowers.zip

### Image loader generator

In [ ]:
## Images PATHs
train_dir = 'data/train/'
validation_dir = 'data/validation/'
test_dir = 'data/test/'

## Image generator instances
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

## Image conversion size
imgSize = 150

## Generator setting (Train)
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to (imgSize x imgSize)
        target_size=(imgSize, imgSize), ## 224
        batch_size=20,
        # Since we use categorical_crossentropy loss, we need categorical labels
        class_mode='categorical')

## Generator setting (Validation)
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(imgSize, imgSize),
        batch_size=20,
        shuffle=False,
        class_mode='categorical')

## Generator setting (test)
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(imgSize, imgSize),
        batch_size=20,
        shuffle=False,
        class_mode='categorical')

# __Actividad__

En base a lo aprendido en clase, genere un modelo de clasificación mediante redes neuronales convolucionales, para predecir el tipo de flor. Utilice el conjunto de entrenamiento y validación de la mejor forma tal de poder definir los hiperparámetros de una red neuronal convolucional (número de capas conv, número de filtros en las capas, tamaño de kernel, funciones de activación, pooling, número de épocas). Una vez que haya definido la mejor configuración de la red neuronal convolucional, evalúe el desempeño de esa red en el conjunto de prueba, en particular entregue el accuracy, precision, recall, y F1-score.

__Ayuda__: Rutina para extraer las etiquetas del generador de test. 

```python
## Reiniciar el generador al prier indice de la imagen
test_generator.reset()

## Extraer las etiquetas (codificadas en One-Hot) del conjunto de test.
i, y_true = 0, []
for _, y in test_generator:
    y_true.append(y)
    i+=1
    if i == 19:
      break
y_true = vstack(y_true)

## Decodificar las etiquetas
y_true = argmax(y_true, axis=1)
```

### Respuesta: